In [1]:
import config
import spotipy
import pickle
import json 
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.preprocessing import StandardScaler
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [2]:
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")

In [6]:
scaler = load(filename = "Model/scaler.pickle")
kmeans = load(filename = "Model/kmeans.pickle")

In [7]:
kmeans

KMeans(n_clusters=7, random_state=100)

In [8]:
def find_features_input(user_input):  
        
    x = sp.search(q=user_input, limit =10, market="GB")['tracks']['items'][1]['external_urls']["spotify"][31:]

    y = sp.audio_features(x)

        
    song_features = pd.DataFrame(y)
    song_id = song_features['id']
    song_features = song_features.drop(['id','type', 'uri', 'track_href', 'analysis_url','time_signature', 'duration_ms'], axis =  1)
        
    
    
    features_scaled = scaler.transform(song_features)
    song_features_scaled = pd.DataFrame(features_scaled, columns = song_features.columns)
    
    return (song_features_scaled)

In [10]:
x = find_features_input('Lose control')

In [11]:
x

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.232378,-0.258988,-0.919317,0.527207,0.791708,-0.443976,-0.866588,-0.646813,-0.658863,-1.186916,0.547238


In [14]:
prediction = kmeans.predict(x)
prediction

array([4], dtype=int32)

In [15]:
ids = pd.read_csv('Data/song_ids_titles_and_artist_names.csv')
audio = pd.read_csv('Data/songs_audio_features_with_clusters.csv')

In [17]:
concat = pd.concat(objs=[ids,audio], axis=1)
concat.drop(columns=['Unnamed: 0'], inplace=True)
concat.to_csv('Data/songs_dataframe_concatenated.csv', index=False)

,title,artist,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,cluster
0,Where Love Lives (Come On In) - Classic Mix,Alison Limerick,0ilZMeXHPKDPjUc4Neu04f,0.708,0.566,11,-10.142,0,0.0362,0.000127,0.033800,0.2900,0.731,121.997,3
1,Passion - Naked Edit,Gat Decor,0ZBPWoRkfZ8SLcDJaFnkUu,0.710,0.625,5,-15.300,0,0.0414,0.004180,0.935000,0.1090,0.183,124.876,2
2,Good Life,Inner City,5sJiLlgQKBL81QCTOkoLB5,0.716,0.783,4,-12.876,0,0.0496,0.172000,0.034600,0.0327,0.952,121.306,3
3,U Don't Know Me (feat. Duane Harden),Armand Van Helden,0TN9m8rp3ZuNi4JwCKSans,0.742,0.756,10,-6.197,1,0.0825,0.021300,0.022700,0.0553,0.810,129.963,4
4,Free - Mood II Swing Remix,Sebastian Gamboa,2FrCwt008k0xqtCFH1VaTZ,0.726,0.907,5,-5.009,0,0.0894,0.000916,0.000521,0.2170,0.498,124.841,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10265,Negativland,NEU!,2x0eC3SRcTwvGJxoXvhfcE,0.378,0.665,2,-11.604,1,0.0352,0.020300,0.858000,0.3180,0.512,110.074,2
10266,Rats,K-Holes,0339v0PaMmnpb7IOdGgnvJ,0.123,0.785,0,-2.621,1,0.0638,0.000001,0.757000,0.3980,0.139,169.221,6
10267,Mes,Murcof,4XjW9fcihP29h7fBAvXlDe,0.527,0.293,0,-16.701,0,0.1320,0.669000,0.841000,0.1040,0.127,105.008,0
10268,Dragonflies to Sew You Up,Prurient,2yr8zjsYtBAeZwpvQYC66k,0.498,0.995,3,-6.110,0,0.0493,0.008020,0.177000,0.2210,0.339,148.241,3
